# ABIA Proyect 1: Hill Climbing

<hr>
<hr>

## Imports

In [55]:
import time
import timeit
import random
import numpy as np
from tkinter import Y
from math import trunc, log
from typing import List, Generator, Set
from search import Problem, hill_climbing

<hr>
<hr>

## File: abia_energia.py

### Constants

In [56]:
# Tipos de cliente
CLIENTE_XG = 0
CLIENTE_MG = 1
CLIENTE_G = 2

# Tipos de central
CENTRAL_A = 0
CENTRAL_B = 1
CENTRAL_C = 2

# Prioridades
GARANTIZADO = 0
NOGARANTIZADO = 1

# Producción por tipo de central
PROD = [[500.0, 250.0], [150.0, 100.0], [90.0, 10.0]]

# Consumo por tipo de cliente
CONSUMOS = [[15.0, 5.0], [3.0, 2.0], [2, 1]]

# Tabla de precios por tipo de cliente
# Primera dimensión: tipo de cliente
# Segunda dimensión, por orden: garantizado, no garantizado, indemnización
PRECIOS = [[40.0, 30.0, 5.0], [50.0, 40.0, 5.0], [60.0, 50.0, 5.0]]

# Tabla de costes
# Primera dimensión: tipo de central
# Segunda dimensión, por orden: coste por producción, coste diario, coste parada
COSTES = [[5, 2000, 1500], [8, 1000, 500], [15, 500, 150]]

# Tabla de pérdidas
# En cada posición, el primer valor es la distancia máxima del rango, el segundo valor es el tanto por uno de pérdida.
PERDIDA = [[10, 0], [25, 0.1], [50, 0.2], [75, 0.4], [1000, 0.6]]
TIPO = [0, 1, 2]
TIPOCL = [0, 1, 2]
TIPOCNT = [0, 1]

<hr>

### Class Central

In [57]:
class Central(object):

    def __init__(self, tipo: int, produccion: float, cx: int, cy: int):
        self.Tipo = tipo
        self.Produccion = produccion
        self.CoordX = cx
        self.CoordY = cy

    def __repr__(self) -> str:
        return f"[Tipo={self.Tipo}|Produccion={self.Produccion}|cx={self.CoordX}|cy={self.CoordY}]"

### Class Centrals

In [58]:
class Centrales(list):
    
    def __init__(self, centrales_por_tipo: List[int], seed: int):
        if len(centrales_por_tipo) != 3:
            raise Exception("Vector Centrales de tamaño incorrecto")
        v_centrales = []
        rand = random.Random(seed)
        for i in range(3):
            for j in range(centrales_por_tipo[i]):
                p = (rand.random() * PROD[i][0]) + PROD[i][1]
                c = Central(TIPO[i], trunc(p), rand.randint(
                    0, 100), rand.randint(0, 100))
                v_centrales.append(c)
        super().__init__(v_centrales)

<hr>

### Class Client

In [59]:
class Cliente(object):
    
    def __init__(self, t: int, cons: float, cont: int, cx: int, cy: int):
        self.Tipo = t
        self.Consumo = cons
        self.Contrato = cont
        self.CoordX = cx
        self.CoordY = cy

    def __repr__(self) -> str:
        return f"[Tipo={self.Tipo}|Consumo={self.Consumo}|Contrato={self.Contrato}|CoordX={self.CoordX}|CoordY={self.CoordY}]"

### Class Clients

In [60]:
class Clientes(list):

    def __init__(self, ncl: int, propc: List[float], propg: float, seed: int):
        if len(propc) != 3:
            raise Exception(
                "Vector proporciones tipos clientes de tamaño incorrecto")
        if (propc[0] + propc[1] + propc[2]) != 1.0:
            raise Exception("Vector proporciones tipos clientes no suma 1")
        if 0.0 > propg > 1.0:
            raise Exception("Proporcion garantizado fuera de limites")

        v_clientes = []
        rand = random.Random(seed + 1)

        for i in range(ncl):
            dice = rand.random()
            if dice < propc[0]:
                rd = 0
            elif dice < (propc[0] + propc[1]):
                rd = 1
            else:
                rd = 2

            dice = rand.random()
            if dice < propg:
                rc = 0
            else:
                rc = 1

            c = (rand.random() * CONSUMOS[rd][0]) + CONSUMOS[rd][1]
            v_clientes.append(
                Cliente(TIPOCL[rd], trunc(c), TIPOCNT[rc], rand.randint(0, 100), rand.randint(0, 100)))
        super().__init__(v_clientes)

<hr>

### Static Class VEnergia

In [61]:
class VEnergia(object):

    @staticmethod
    def tarifa_cliente_garantizada(tipo: int) -> float:
        
        """ Retorna la tarifa de un cliente con servicio garantizado. """

        if tipo < 0 or tipo > 2:
            raise Exception("Tipo fuera de rango")
        else:
            return PRECIOS[tipo][0]

    @staticmethod
    def tarifa_cliente_no_garantizada(tipo: int) -> float:
        
        """ Retorna la tarifa de un cliente con servicio no garantizado. """
        
        if tipo < 0 or tipo > 2:
            raise Exception("Tipo fuera de rango")
        else:
            return PRECIOS[tipo][1]

    @staticmethod
    def tarifa_cliente_penalizacion(tipo: int) -> float:
        
        """ Retorna la penalización por servir a un cliente con servicio no garantizado. """
        
        if tipo < 0 or tipo > 2:
            raise Exception("Tipo fuera de rango")
        else:
            return PRECIOS[tipo][2]

    @staticmethod
    def prices_production_mw(tipo: int) -> float:
        
        """ Retorna el coste de producción por MW para una central de un tipo. """
        
        if tipo < 0 or tipo > 2:
            raise Exception("Tipo fuera de rango")
        else:
            return COSTES[tipo][0]

    @staticmethod
    def daily_cost(tipo: int) -> float:
        
        """ Retorna el coste de una central en marcha segun su tipo. """
        
        if tipo < 0 or tipo > 2:
            raise Exception("Tipo fuera de rango")
        else:
            return COSTES[tipo][1]

    @staticmethod
    def stop_cost(tipo: int) -> float:
        
        """ Retorna el coste de una central en parada segun su tipo. """
        
        if tipo < 0 or tipo > 2:
            raise Exception("Tipo fuera de rango")
        else:
            return COSTES[tipo][2]

    @staticmethod
    def loss(distancia: float) -> float:
        
        """ Retorna la perdida (en tanto por uno) segun la distancia entre central y cliente. """
        
        i = 0
        while distancia > PERDIDA[i][0]:
            i = i + 1
        return PERDIDA[i][1]

<hr>
<hr>

## File: abia_energy_problem.py

### Class Problem Parameters

Esta clase es la misma desde que se crea, por tanto, ¿podríamos evitarnos pasarla por cada copia? 

In [62]:
class ProblemParameters(object):
    def __init__(self, clients_vector: Clientes, power_plants_vector: Centrales) -> None:
        self.clients_vector = clients_vector
        self.power_plants_vector = power_plants_vector

    def __repr__(self):
        return f"clients_vector={self.clients_vector}\n\npower_plants_vector={self.power_plants_vector})"

<hr>

### Operators

In [63]:
class Operator(object):
    pass

class MoveClient(Operator):
    def __init__(self, id_client, id_destination_PwP):
        self.id_client = id_client
        self.id_destination_PwP = id_destination_PwP

    def __repr__(self) -> str:
        return f"Client {self.id_client} has been moved to power plant {self.id_destination_PwP}"

class SwapClients(Operator):
    def __init__(self, id_client1, id_client2):
        self.id_client1 = id_client1
        self.id_client2 = id_client2

    def __repr__(self) -> str:
        return f"Swap between Client {self.id_client1} and Client {self.id_client2}"

class RemoveNGClient(Operator):
    def __init__(self, id_client: int):
        self.id_client = id_client

    def __repr__(self) -> str:
        return f"Client {self.id_client} has been removed"

class TurnOnPowerPlant(Operator):
    def __init__(self, id_pp: int):
        self.id_pp = id_pp

    def __repr__(self) -> str:
        return f"Power Plant {self.id_pp} has been turned on"

<hr>

### Our methods

Se podria meter en VEnergia?

In [64]:
def distance(obj1, obj2):
    x = obj1.CoordX - obj2.CoordX
    y = obj1.CoordY - obj2.CoordY
    return (x**2 + y ** 2)**(1/2)

def electry_supplied_to_client(client: Cliente, power_plant: Central) -> int:
    dist = distance(client, power_plant)
    return client.Consumo * (1 + VEnergia.loss(dist))

# def calculate_entropy(params:ProblemParameters, id, remain):
#         max_prod = params.power_plants_vector[id].Produccion
#         occupancy = 1 - (remain/max_prod)
#         if occupancy > 0:
#             return - (occupancy * log(occupancy))
#         return 0


<hr>

### Our constants

In [65]:
NONPOWERPLANT = -1

### Class State Representation

In [66]:
class StateRepresentation(object):
    
    def __init__(self, 
                 params: ProblemParameters, 
                 c_pp: List[int], 
                 remain: List[float], 
                 consum: List[List[float]], 
                 gain: float,
                 entropy:float,
                 prices: List[List[float]]) -> None:

        self.params = params
        self.c_pp   = c_pp
        self.remain = remain
        self.consum = consum
        self.prices = prices
        self.gain   = gain
        self.entropy = entropy

    def copy(self):
        return StateRepresentation(self.params, 
                                   self.c_pp.copy(), 
                                   self.remain.copy(), 
                                   self.consum, 
                                   self.gain,
                                   self.entropy,
                                   self.prices)

    def __repr__(self) -> str:
        s_c = 0
        for i in self.c_pp:
            if i != NONPOWERPLANT: s_c += 1
        return f"Serviced clietns = {s_c} of {len(self.c_pp)} \nGains: {self.gain_heuristic()} \n"
        
    def generate_actions(self) -> Generator[Operator, None, None]:
        num_c  = len(self.params.clients_vector)
        num_pp = len(self.params.power_plants_vector)

        for id_pp in range(num_pp):
            
            # TurnOnPowerPlant
            if self.remain[id_pp] == self.params.power_plants_vector[id_pp].Produccion:
                yield TurnOnPowerPlant(id_pp)

            # Move Client
            else:
                for id_c in range(num_c):
                    if id_pp == self.c_pp[id_c]:
                        continue

                    c_consum = self.consum[id_pp][id_c]
                    if c_consum < self.remain[id_pp]:
                        yield MoveClient(id_c, id_pp)
        
        for id_c1 in range(num_c):
            #Remove client
            if self.params.clients_vector[id_c1].Tipo == NOGARANTIZADO:
                 yield RemoveNGClient(id_c1)

            # Swap Client
            if id_c1 != num_c - 1:
                for id_c2 in range(id_c1 + 1, num_c):
                    id_pp1 = self.c_pp[id_c1]
                    id_pp2 = self.c_pp[id_c2]

                    if id_pp1 == id_pp2 or id_pp1 == NONPOWERPLANT or id_pp2 == NONPOWERPLANT:
                        continue

                    consum_c1_pp1 = self.consum[id_pp1][id_c1]
                    consum_c1_pp2 = self.consum[id_pp2][id_c1]
                    consum_c2_pp1 = self.consum[id_pp1][id_c2]
                    consum_c2_pp2 = self.consum[id_pp2][id_c2]

                    remain_pp1 = self.remain[id_pp1]
                    remain_pp2 = self.remain[id_pp2]

                    if consum_c2_pp1 - consum_c1_pp1 < remain_pp1 and consum_c1_pp2 - consum_c2_pp2 < remain_pp2:
                        yield SwapClients(id_c1, id_c2)

    def apply_action(self, action: Operator):
        new_state = self.copy()

        if isinstance(action, MoveClient):
            id_c   = action.id_client
            id_pp1 = self.c_pp[id_c] 
            pp1    = self.params.power_plants_vector[id_pp1]
            id_pp2 = action.id_destination_PwP
            pp2    = self.params.power_plants_vector[id_pp2]

            new_state.c_pp[id_c] = id_pp2

            if id_pp1 != NONPOWERPLANT:
                # new_state.entropy -= calculate_entropy(self.params, id_pp1, self.remain[id_pp1])
                new_state.remain[id_pp1] += self.consum[id_pp1][id_c]
                # new_state.entropy += calculate_entropy(self.params, id_pp1, new_state.remain[id_pp1])
                
                # Podria ser un self.remain??
                if new_state.remain[id_pp1] == pp1.Produccion:
                    new_state.gain += self.prices[1][id_pp1] - self.prices[0][id_pp1]
    
            else:
                new_state.gain += self.prices[2][id_c]
            
            # Podria ser un self.remain??
            # if new_state.remain[id_pp2] == pp2.Produccion:
            #     new_state.gain -= self.prices[1][id_pp2] + self.prices[0][id_pp2]
            # new_state.entropy -= calculate_entropy(self.params, id_pp2, self.remain[id_pp2])
            new_state.remain[id_pp2] -= self.consum[id_pp2][id_c]
            # new_state.entropy += calculate_entropy(self.params, id_pp2, new_state.remain[id_pp2])

        elif isinstance(action, SwapClients):
            id_c1  = action.id_client1
            id_c2  = action.id_client2
            id_pp1 = self.c_pp[id_c1]
            id_pp2 = self.c_pp[id_c2]

            new_state.c_pp[id_c1] = id_pp2
            new_state.c_pp[id_c2] = id_pp1

            # new_state.entropy -= calculate_entropy(self.params, id_pp1, self.remain[id_pp1])
            # new_state.entropy -= calculate_entropy(self.params, id_pp2, self.remain[id_pp2])

            new_state.remain[id_pp1] += self.consum[id_pp1][id_c1] - self.consum[id_pp1][id_c2]
            new_state.remain[id_pp2] += self.consum[id_pp2][id_c2] - self.consum[id_pp2][id_c1]

            # new_state.entropy += calculate_entropy(self.params, id_pp1, new_state.remain[id_pp1])
            # new_state.entropy += calculate_entropy(self.params, id_pp2, new_state.remain[id_pp2])

        elif isinstance(action, TurnOnPowerPlant):
            new_state.gain -= self.prices[1][action.id_pp] + self.prices[0][action.id_pp]
    
        elif isinstance(action, RemoveNGClient):
            id_c = action.id_client

            new_state.c_pp[id_c] = -1

            id_pp = self.c_pp[id_c]

            # new_state.entropy -= calculate_entropy(self.params, id_pp, self.remain[id_pp])

            new_state.remain[id_pp] += self.consum[id_pp][id_c]

            # new_state.entropy += calculate_entropy(self.params, id_pp, new_state.remain[id_pp])

        return new_state

    def gain_heuristic(self) -> float:
        return self.gain

    def entropy_heuritic(self) -> float:
        return InitialState.calculate_total_entropy(self.params, self.remain)

    def combined_heuristic(self) -> float:
        return self.gain_heuristic() + self.entropy_heuritic()

<hr>

### Generation of the initial state

In [67]:
class InitialState(object):

    @staticmethod
    def simple_state(params) -> StateRepresentation:
        remain, consum = InitialState.remain_consum(params)
        
        c_pp  = []
        pp    = list(range(len(params.power_plants_vector)))
        id_pp = pp.pop()
        for id_client, client in enumerate(params.clients_vector):
            if client.Contrato == NOGARANTIZADO:
                c_pp.append(NONPOWERPLANT)
                continue

            c_consum = consum[id_pp][id_client]
            while True:
                if c_consum < remain[id_pp]:
                    c_pp.append(id_pp)
                    remain[id_pp] -= c_consum
                    break
                id_pp = pp.pop()

        c_pp = np.array(c_pp)
        
        gain, prices = InitialState.calculate_gain(params, c_pp, remain)
        entropy = InitialState.calculate_total_entropy(params, remain)
        return StateRepresentation(params, c_pp, remain, consum, gain, entropy, prices)
    
    @staticmethod
    def simple_state2(params, worst=False, invert= False) -> StateRepresentation:
        c_pp   = []
        num_c  = len(params.clients_vector)
        num_pp = len(params.power_plants_vector)
        id_pp = 0 if not invert else len(params.power_plants_vector)
        id_c = cycle= 0
        MAX_NUM_CYCLE = 15
        def next_pp():
            nonlocal id_pp, cycle
            print(id_pp)
            id_pp += 1 if not invert else -1
            if invert and id_pp != 0 or not invert and id_pp == num_pp:
                return

            id_pp = 0
            cycle += 1
            if cycle == MAX_NUM_CYCLE:
                raise Exception("Sorry, max number of cycles did it")

        remain, consum = InitialState.remain_consum(params)
        

        while id_c < num_c:
            if params.clients_vector[id_c].Tipo == NOGARANTIZADO:
                c_pp.append(NONPOWERPLANT)

            elif remain[id_pp] > consum[id_pp][id_c]:
                remain[id_pp] -= consum[id_pp][id_c]
                c_pp.append(id_pp)
            else:
                next_pp()
                continue

            if worst:
                next_pp()
            id_c += 1 

        c_pp = np.array(c_pp)

        gain, prices = InitialState.calculate_gain(params, c_pp, remain)

        entropy = InitialState.calculate_total_entropy(params, remain)

        return StateRepresentation(params, c_pp, remain, consum, gain, entropy, prices)


    @staticmethod
    def generate_simple_initial_state2(params: ProblemParameters, worst=False):
        remaining_energies, real_consumption = InitialState.remain_consum(params)
        client_power_plant = list()
        numClients = len(params.clients_vector)
        numPwP = len(params.power_plants_vector)
        id_client = 0
        id_PwP = 0
        cycle = 0
        MAX_NUM_CYCLE = 15 *   # Max num
        while id_client < numClients:
            if params.clients_vector[id_client].Tipo == NOGARANTIZADO:
                client_power_plant.append(-1)

            elif remaining_energies[id_PwP] > real_consumption[id_PwP][id_client]:
                remaining_energies[id_PwP] -= real_consumption[id_PwP][id_client]
                client_power_plant.append(id_PwP)
            else:
                # Si no se han cumplido ninguna de las dos condiciones, el cliente no es añadido a la central
                # Por eso pasamos a mirar la siguiente central
                id_PwP += 1

                # Si nos exedemos del número de centrales, volvemos a poner este valor a cero
                if id_PwP == numPwP:
                    id_PwP = 0
                    cycle += 1
                    if cycle == MAX_NUM_CYCLE:
                        raise Exception("Sorry, max number of cycles did it")
                continue

            if worst:
                id_PwP += 1
                if id_PwP == numPwP:
                    id_PwP = 0
                    cycle += 1
                    if cycle == MAX_NUM_CYCLE:
                        raise Exception("Sorry, max number of cycles did it")
            id_client += 1

        client_power_plant = np.array(client_power_plant)
        gain, prices = InitialState.calculate_gain(params, remaining_energies, client_power_plant)
        entropy = InitialState.calculate_total_entropy(params, remaining_energies)


        return StateRepresentation(params, client_power_plant, remaining_energies, real_consumption, gain, entropy, prices)
        

    @staticmethod
    def complex_state(params: ProblemParameters, n: int, useNG: bool) -> StateRepresentation:
        remaining_energies, real_consumption = InitialState.remain_consum(params)

        client_power_plant = [-1] * len(params.clients_vector)
        div = 100 // n

        zones = [[[] for i in range(n)]for _ in range(n)]

        def assign_power_plant(client_vector, isGuaranteed):
            for id, client in client_vector:
                X = min(client.CoordX // div, n - 1)
                Y = min(client.CoordY // div, n - 1)

                Z1 = 0
                Z2 = 0
                Z3 = 0
                dist = 0
                next_moveX = []
                next_moveY = []

                while True:
                    if len(zones[Y + Z1][X + Z2]) > 0:
                        PwP = zones[Y + Z1][X + Z2][Z3]
                        consum = real_consumption[PwP][id]
                        if consum < remaining_energies[PwP]:
                            client_power_plant[id] = PwP
                            remaining_energies[PwP] -= consum
                            break
                        # Se van mirando las centrales de cada zona hasta que no quedan más
                        elif Z3 < len(zones[Y + Z1][X + Z2]) - 1:
                            Z3 += 1
                            continue

                    if len(next_moveX) == 0:

                        dist += 1

                        # All region watched
                        if dist >= n:
                            if isGuaranteed:
                                raise Exception(
                                    "Too much client than power plants")
                            else:
                                break

                        lim_left = -dist if X - dist >= 0 else 0
                        lim_down = -dist if Y - dist >= 0 else 0
                        lim_right = dist if X + dist < n else (n - 1) - X
                        lim_up = dist if Y + dist < n else (n - 1) - Y

                        # We will use pop, so the order is inversed that we are doing here

                        moves_axis_X = list(range(lim_left, lim_right + 1))
                        moves_axis_Y = list(range(lim_down, lim_up + 1))

                        # Go right
                        next_moveX += moves_axis_X
                        next_moveY += [lim_down] * len(moves_axis_X)

                        # Go up
                        next_moveX += [lim_right] * len(moves_axis_Y)
                        next_moveY += moves_axis_Y

                        # Go left
                        next_moveX += reversed(moves_axis_X)
                        next_moveY += [lim_up] * len(moves_axis_X)

                        # Go down
                        next_moveX += [lim_left] * len(moves_axis_Y)
                        next_moveY += reversed(moves_axis_Y)

                    else:
                        Z3 = 0
                        Z1 = next_moveY.pop()
                        Z2 = next_moveX.pop()

        for id, PwP in enumerate(params.power_plants_vector):
            # podría passar que en algún caso muy extremo CoordX = 100 or CoordY
            # En ese caso queremos que su X or Y sea 9 y no 10
            X = min(PwP.CoordX // div, n - 1)
            Y = min(PwP.CoordY // div, n - 1)
            zones[Y][X].append(id)

        G_clients = filter(
            lambda x: x[1].Contrato != NOGARANTIZADO, enumerate(params.clients_vector))

        assign_power_plant(G_clients, True)

        if useNG:
            NG_clients = filter(
                lambda x: x[1].Contrato == NOGARANTIZADO, enumerate(params.clients_vector))
            assign_power_plant(NG_clients, False)

        gain, prices = InitialState.calculate_gain(params, client_power_plant, remaining_energies)
        entropy = InitialState.calculate_total_entropy(params, remaining_energies)

        return StateRepresentation(params, client_power_plant, remaining_energies, real_consumption, gain, entropy, prices)
    
    
    @staticmethod
    def remain_consum(params):
        remain = []
        consum = []

        for pp in params.power_plants_vector:
            remain.append(pp.Produccion)
            row = []
            for client in params.clients_vector:
                c_consum = electry_supplied_to_client(client, pp)
                row.append(c_consum)
            consum.append(row)

        remain = np.array(remain)
        consum = np.array(consum)
        return remain, consum

    @staticmethod
    def calculate_gain(params, c_pp, remain) -> float:
        gain = 0
        prices = [[], [], []]

        for id_pp, pp in enumerate(params.power_plants_vector):
            prices[0].append(VEnergia.stop_cost(pp.Tipo))
            prices[1].append(VEnergia.daily_cost(pp.Tipo) + VEnergia.prices_production_mw(pp.Tipo) * pp.Produccion)

            if pp.Produccion == remain[id_pp]: 
                gain -= prices[0][id_pp]
            else:
                gain -= prices[1][id_pp]

        for id_c, c in enumerate(params.clients_vector):
            if c.Contrato == 0:   
                gain += c.Consumo * VEnergia.tarifa_cliente_garantizada(c.Tipo)
            else:                 
                gain += c.Consumo * VEnergia.tarifa_cliente_no_garantizada(c.Tipo)

            prices[2].append(c.Consumo * VEnergia.tarifa_cliente_penalizacion(c.Tipo))
            if c_pp[id_c] == NONPOWERPLANT:  gain -= prices[2][id_c]

        return gain, prices

    @staticmethod
    def calculate_total_entropy(params: ProblemParameters, remaining_energies):
        total_entropy = 0
        for id, remain in enumerate(remaining_energies):
            max_prod = params.power_plants_vector[id].Produccion
            occupancy = 1 - (remain/max_prod)
            if occupancy > 0:
                total_entropy +=  - (occupancy * log(occupancy))            
        return -total_entropy
            

<hr>

### Class of the Problem

In [68]:
class EnergyProblem(Problem):
    def __init__(self, initial_state: StateRepresentation):
        self.times = [0, 0, 0]
        self.iterations = [0, 0, 0]
        super().__init__(initial_state)

    def actions(self, state: StateRepresentation) -> Generator[Operator, None, None]:

        # print(state.remain)
        # print(sum(state.remain))
        start   = time.time()
        actions = state.generate_actions()
        end     = time.time()
        self.times[0] += end - start
        self.iterations[0] += 1
        return actions

    def result(self, state: StateRepresentation, action: Operator) -> StateRepresentation:
        start   = time.time()
        result  = state.apply_action(action)
        end     = time.time()
        self.times[1] += end - start
        self.iterations[1] += 1
        return result

    def value(self, state: StateRepresentation) -> float:
        start   = time.time()
        value   = state.gain_heuristic()
        end     = time.time()
        self.times[2] += end - start
        self.iterations[2] += 1
        return value

    def goal_test(self, state: StateRepresentation) -> bool:
        return False

<hr>
<hr>

## Execution of the program

In [69]:
clientes       = Clientes(ncl=1000, propc=[0.25, 0.3, 0.45], propg=0.75, seed=1234)
centrales      = Centrales(centrales_por_tipo=[5, 10, 25], seed=1234)
parametros     = ProblemParameters(clients_vector=clientes, power_plants_vector=centrales)
estado_inicial = InitialState.simple_state2(parametros)
problem        = EnergyProblem(estado_inicial)


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


Exception: Sorry, max number of cycles did it

In [ ]:
print(estado_inicial)

start     = time.time()
ejecucion = hill_climbing(problem)
end       = time.time()

print(ejecucion)

print(f"Time taken: {end - start}s")

Serviced clietns = 754 of 1000 
Gains: 115090.0 

Serviced clietns = 773 of 1000 
Gains: 116285.0 

Time taken: 382.8175446987152s


<hr>

### Analizing problems

Hemos añadido codigo en la clase EnergyProblem que cronometra el tiempo de cada ejecución de las funciones que tenemos y cuenta cuantas veces se ejecuta.

In [ ]:
print(f"Generate actions: {problem.times[0]}s, {problem.iterations[0]} times")
print(f"Apply action: {problem.times[1]}s, {problem.iterations[1]} times")
print(f"Heuristic: {problem.times[2]}s, {problem.iterations[2]} times")

Generate actions: 0.0s, 27 times
Apply action: 18.241452932357788s, 2086887 times
Heuristic: 0.5562362670898438s, 2086941 times


Tenemos que reducir el tiempo de ejecución de <i style="color: #ffa530">apply_actions()</i> ya que dura demasiado.

In [ ]:
print(timeit.timeit(lambda: estado_inicial.generate_actions(), number=1))
print(timeit.timeit(lambda: estado_inicial.apply_action(MoveClient(0, 1)), number=1))
print(timeit.timeit(lambda: estado_inicial.apply_action(SwapClients(0, 1)), number=1))
print(timeit.timeit(lambda: estado_inicial.gain_heuristic(), number=1))

8.199999683711212e-06
0.00010280000014972757
5.880000026081689e-05
3.099999958067201e-06
